# Install the required libraries

In [2]:
!pip install -q streamlit
!pip install -q peft
!pip install -q bitsandbytes

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.7/8.7 MB 49.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.3/207.3 kB 13.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 51.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 82.9/82.9 kB 7.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 5.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 296.4/296.4 kB 6.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 137.5/137.5 MB 7.0 MB/s eta 0:00:00


# Mount your drive

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# Load the saved model from your drive

In [1]:
import streamlit as st
from transformers import AutoTokenizer
from peft import AutoPeftModelForCausalLM

# Load the model and tokenizer
model = AutoPeftModelForCausalLM.from_pretrained(
    "/content/drive/MyDrive/Nebula9.ai",
    load_in_4bit = True,
)
tokenizer = AutoTokenizer.from_pretrained("/content/drive/MyDrive/Nebula9.ai")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
The `load_in_4bit` and `load_in_8bit` arguments are deprecated and will be removed in the future versions. Please, pass a `BitsAndBytesConfig` object in `quantization_config` argument instead.
Unused kwargs: ['_load_in_4bit', '_load_in_8bit', 'quant_method']. These kwargs are not used in <class 'transformers.utils.quantization_config.BitsAndBytesConfig'>.
/usr/local/lib/python3.10/dist-packages/transformers/quantizers/auto.py:174: UserWarning: You passed `quantization_config` 

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

# Write the code for your app which will be hosted through streamlit

In [1]:
%%writefile app.py

import streamlit as st
from transformers import AutoTokenizer
from peft import AutoPeftModelForCausalLM

# Load the model and tokenizer
model = AutoPeftModelForCausalLM.from_pretrained(
    "/content/drive/MyDrive/Nebula9.ai",
    load_in_4bit = True,
)
tokenizer = AutoTokenizer.from_pretrained("/content/drive/MyDrive/Nebula9.ai")

# Define the prompt template
prompt = """Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
Based on the dietary tags and available ingredients, generate a detailed recipe that includes the following:
1. Recipe Name
2. Recipe Directions
3. Nutritional Values (Calories, Protein, Fat, Sodium)

Ensure that the recipe fits the dietary restrictions specified by the tags and make adjustments to the ingredients if necessary to meet those requirements.

### Input:
{}

### Response:
{}
"""

# Function to generate recipe based on user inputs
def generateRecipe(model, tokenizer, dietary_tag, ingredients):
    inputs = tokenizer(
    [
        prompt.format(f"Dietary tags: {dietary_tag}.\nAvailable ingredients: {ingredients}", "")

    ], return_tensors = "pt").to("cuda")



    # Generate the output
    outputs = model.generate(input_ids = inputs.input_ids, attention_mask = inputs.attention_mask, max_new_tokens = 1024, pad_token_id = tokenizer.eos_token_id)


    # Decode the output and process the response
    script = tokenizer.batch_decode(outputs, skip_special_tokens=True)
    response = script[0].split("### Response:")[1].strip()
    return response

# Streamlit app setup
st.set_page_config(page_title="Recipe Generator", page_icon="🍽️")
st.title("Recipe Generator")
st.write("Generate a recipe based on dietary restrictions and available ingredients.")

# Dietary tag selection
dietary_tag = st.selectbox(
    "Select Dietary Tag:",
    ["vegan", "low carb", "non-veg", "low calorie", "keto", "paleo"]
)

# Ingredients input
ingredients = st.text_area(
    "Enter Available Ingredients:",
    placeholder="e.g., 4 cups low-sodium vegetable stock, 1 cup dried brown lentils, 1 large carrot, chopped, etc.",
    height=150
)

# Generate recipe button
if st.button("Generate Recipe"):
    with st.spinner("Generating recipe..."):
        try:
            recipe = generateRecipe(model, tokenizer, dietary_tag, ingredients)
            st.write("### Generated Recipe:")
            st.success(recipe)
        except Exception as e:
            st.error(f"An error occurred: {e}")


Overwriting app.py


# Run the below cell

In [3]:
!npm install localtunnel


up to date, audited 23 packages in 631ms

3 packages are looking for funding
  run `npm fund` for details

2 moderate severity vulnerabilities

To address all issues (including breaking changes), run:
  npm audit fix --force

Run `npm audit` for details.


# Final step run the below line and you will get a link where you have to pass your IP Address (ex-34.125.53.120) as password then you will be redirected to your streamlit application


In [ ]:
!streamlit run app.py &>/content/logs.txt & npx localtunnel --port 8501 & curl ipv4.icanhazip.com

34.125.53.120
your url is: https://fine-wings-fail.loca.lt
